In [1]:
from selenium import webdriver
import pandas as pd
import time

In [2]:
def get_date_df(filename='flood.xlsx'):
    raw_data = pd.read_excel(filename, skiprows=2, header=None, usecols=[0, 1, 2, 3])

    raw_data.columns = ['flood_id', 'year', 'month', 'day']
    raw_data.set_index(['flood_id'], inplace=True)

    date_df = pd.to_datetime(raw_data).dt.strftime('%Y-%m-%d') \
                .groupby('flood_id').agg(['first', 'last'])
    
    return date_df

In [3]:
FLOOD_DATE = get_date_df('../flood.xlsx')
FLOOD_DATE.head()

,first,last
flood_id,,
1,2006-07-10,2006-07-19
2,2006-07-25,2006-07-30
3,2007-08-03,2007-08-07
4,2007-09-14,2007-09-15
5,2008-07-23,2008-07-27


In [4]:
driver = webdriver.Chrome('./chromedriver.exe')
# driver = webdriver.Firefox('./geckodriver.exe')
driver.get('https://www.water.or.kr/realtime/sub01/sub01/dam/hydr.do?seq=1408&p_group_seq=1407&menu_mode=2')
driver.implicitly_wait(5)
driver.refresh()

In [5]:
# 다목적댐 제외한 댐 이름 리스트에 저장하기
dam_ids = []

dam_box = driver.find_element_by_xpath('//*[@id="sub_container"]/div[2]/div[2]/div[3]/div/div[1]/div[2]') \
                .find_elements_by_tag_name('ul')
                
del dam_box[2]

for dam_list in dam_box:
    dams = dam_list.find_elements_by_tag_name('li')
    del dams[0]
    
    for dam in dams:
        dam_ids.append(dam.get_attribute('id'))

In [6]:
print(dam_ids)
len(dam_ids)

['1012110', '1006110', '2001110', '2002110', '2015110', '2018110', '2021110', '2008101', '2010101', '2004101', '2002111', '2012101', '1003110', '3008110', '3203110', '3001110', '4001110', '4007110', '4104610', '3303110', '5101110', '1001210', '1302210', '2012210', '2101210', '2403201', '2021210', '2201231', '2201220', '2201230', '2301210', '2503210', '2503220', '4105210', '5002201', '1009710', '1021701', '1022701', '1007701', '5001701', '5003701', '2022510']


42

In [22]:
from selenium.webdriver.support.ui import Select

period_select = Select(driver.find_element_by_xpath('//*[@id="formtab1"]/fieldset/div/div/div[2]/span/select[1]'))
period_select.select_by_visible_text('시간별')

In [25]:
dam_select = Select(driver.find_element_by_xpath('//*[@id="formtab1"]/fieldset/div/div/div[2]/span/select[2]'))

first_date = driver.find_element_by_xpath('//*[@id="startDate1"]')
last_date = driver.find_element_by_xpath('//*[@id="endDate1"]')

search_btn = driver.find_element_by_xpath('//*[@id="formtab1"]/fieldset/div/a')

table = driver.find_element_by_xpath('//*[@id="grid"]/div/div[1]/div[2]/div[2]/div/div[1]/table')
tbody = table.find_element_by_tag_name('tbody')

driver.implicitly_wait(1)

LAST_FLOOD = 26

writer = pd.ExcelWriter('other_dams.xlsx', engine='xlsxwriter')


# 댐마다
for dam_id in dam_ids[:]:
    dam_select.select_by_value(dam_id)
    dam_name = dam_select.first_selected_option.text
    
    dam_df_list = []
    
    # 홍수사상별
    for flood_id in range(1, LAST_FLOOD+1):
        first_date.clear()
        first_date.send_keys(FLOOD_DATE.at[flood_id, 'first'])
        last_date.clear()
        last_date.send_keys(FLOOD_DATE.at[flood_id, 'last'])
        
        search_btn.click()
        time.sleep(1)
                
        if not tbody.find_elements_by_xpath(".//*"):
            break
        
        driver.find_element_by_class_name('tui-last').click()
        last_page = driver.find_element_by_css_selector('strong.tui-page-btn.tui-is-selected.tui-last-child').text
        last_page = int(last_page)
        driver.implicitly_wait(1.5)


        flood_df_list = []
        
        driver.find_element_by_class_name('tui-first').click()
        flood_df_list.append(pd.read_html(table.get_attribute("outerHTML"))[0])
        driver.implicitly_wait(1.5)
        
        # 출력 페이지별
        for page in range(2, last_page+1):
            driver.find_element_by_class_name('tui-next').click()
            flood_df_list.append(pd.read_html(table.get_attribute("outerHTML"))[0])
            driver.implicitly_wait(1.5)
        
        flood_df = pd.concat(flood_df_list)
        flood_df.insert(0, 'flood_id', flood_id)
        flood_df = flood_df[::-1]
        dam_df_list.append(flood_df)
        
        print(flood_id, end=' ')
        
    
    if dam_df_list:
        dam_df = pd.concat(dam_df_list)#.reset_index(drop=True)
        dam_df.columns = ['flood_id', 'period', 'waterlevel', 'reservoir', 'storage_rate', 'rainfall', 'inflow', 'discharge']
        
        dam_df.to_excel(writer, sheet_name=dam_name, index=False)
        print(dam_name, '완료')
    
writer.save()
writer.close()

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 운문댐 완료


C:\Users\qaws1\anaconda3\lib\site-packages\xlsxwriter\workbook.py:336: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


> 2010-09-18 17시 자료가 없는 '사연댐'은 사후적으로 제거함

In [29]:
# 브라우저 닫기
driver.close()